## Entrenamiento inicial del modelo y búsqueda de hiperparámetros

Para realizar este notebook se utilizó como base el ejemplo presentado en el curso para el dataset "Heart Disease", incluyendo algunas modificaciones para adaptarlo al nuevo dataset "Rain in Australia".

El objetivo de este notebook es realizar experimentos sobre el modelo para luego llevarlo ambiente de producción en caso de que supere ciertas pruebas y además sea aprobado por quien corresponda en la organización.

Para que el entrenamiento funcione localmente en un tiempo razonable se definió la variable *n_samples_train* que permite indicar qué tamaño tiene el subconjunto a entrenar, de forma de no utilizar todos los datos. Esto debe definirse sólo a modo de prueba para aquellos casos en que no se cuente con los recursos computacionales necesarios para procesar el set completo. En caso de que *n_samples_train* sea 0 se ignorará este parámetro.

In [1]:
import awswrangler as wr

import mlflow
from mlflow import MlflowClient, MlflowException
from mlflow.models import infer_signature
import optuna

from sklearn.tree import DecisionTreeClassifier

from mlflow_aux import get_or_create_experiment
from optuna_aux import champion_callback, objective
from plots import plot_correlation_with_target, plot_information_gain_with_target, plot_feature_correlation

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import random
import datetime

%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

# Cantidad de samples a utilizar para el entrenamiento
n_samples_train = 1000

c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [4]:
# Se setea URL de seguimiento para MLflow
mlflow_server = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_server)

Se cargan los datos de entrenamiento y validación desde el bucket S3. A este punto se asume que ya corrió sin errores el DAG *etl_process_rain_australia*, haciendo las transformaciones necesarias de los datos.

In [7]:
# Se cargan los conjuntos de entrenamiento y validación.

X_train =  wr.s3.read_csv("s3://data/final/train/tumors_X_train.csv")
y_train =  wr.s3.read_csv("s3://data/final/train/tumors_y_train.csv")

X_test =  wr.s3.read_csv("s3://data/final/test/tumors_X_test.csv")
y_test =  wr.s3.read_csv("s3://data/final/test/tumors_y_test.csv")

NoFilesFound: No files Found on: s3://data/final/train/tumors_X_train.csv.

In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106644 entries, 0 to 106643
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MinTemp          106644 non-null  float64
 1   MaxTemp          106644 non-null  float64
 2   Rainfall         106644 non-null  float64
 3   Evaporation      106644 non-null  float64
 4   Sunshine         106644 non-null  float64
 5   WindGustSpeed    106644 non-null  float64
 6   WindSpeed9am     106644 non-null  float64
 7   WindSpeed3pm     106644 non-null  float64
 8   Humidity9am      106644 non-null  float64
 9   Humidity3pm      106644 non-null  float64
 10  Pressure9am      106644 non-null  float64
 11  Pressure3pm      106644 non-null  float64
 12  Cloud9am         106644 non-null  float64
 13  Cloud3pm         106644 non-null  float64
 14  Temp9am          106644 non-null  float64
 15  Temp3pm          106644 non-null  float64
 16  RainToday        106644 non-null  floa

Previo a la elaboración del DAG *etl_process_rain_australia* ya se realizó un análisis y limpieza del dataset en el que, entre otras acciones, se determinó las correlación entre las variables de entrada y de éstas con la variable objetivo. De todas formas, es útil registrar esta información para observar en el tiempo los posibles cambios que se puedan producir.

In [19]:
# Se obtienen los gráficos correspondientes a las correlaciones
correlation_plot = plot_correlation_with_target(X_train, y_train, target_col='target')
information_gain_plot = plot_information_gain_with_target(X_train, y_train, target_col='target')
features_correlation_plot = plot_feature_correlation(X_train)

Las diferentes ejecuciones de entrenamiento quedarán registradas en el experimento "Rain in Australia". El tuneo de hiperparámetros se mostrará como un *run* dentro de este experimentos que a su vez contendrá *runs* hijos correspondientes a cada prueba (*trial*) realizada.

In [20]:
experiment_id = get_or_create_experiment("Cancer de mama")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


Tal como se comentó al comienzo, en caso de estar definida la variable *n_samples_train* se tomará un subconjunto de datos. La idea de utilizar esto es sólo para pruebas en equipos con bajos recursos computacionales.

In [21]:
if n_samples_train > 0:

    indices = random.sample(range(len(X_train)), n_samples_train)

    X_train = X_train.iloc[indices].reset_index(drop=True)
    y_train = y_train.iloc[indices].reset_index(drop=True)

Se realiza la optimización usan Optuna. Además, como se indicó anteriormente, se reutilizaron las librerías del ejemplo proporcionado en el curso dado que simplifican mucho la tarea y proporcionan una base sólida para futuras mejoras.

In [22]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna
    study = optuna.create_study(direction="maximize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=250, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Cancer de mama",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["classifier"] == "SVC_linear":
        model = SVC(C=study.best_params["svc_c"], kernel='linear', gamma='scale')
    elif study.best_params["classifier"] == "SVC_poly":
        model = SVC(C=study.best_params["svc_c"], kernel='poly', 
                    gamma='scale', degree=study.best_params["svc_poly_degree"])
    elif study.best_params["classifier"] == "SVC_rbf":
        model = SVC(C=study.best_params["svc_c"], kernel='rbf', gamma='scale')
    elif study.best_params["classifier"] == "DecisionTreeClassifier":
        model = DecisionTreeClassifier(max_depth=study.best_params["tree_max_depth"])
    else:
        model = RandomForestClassifier(max_depth=study.best_params["rf_max_depth"], 
                                       n_estimators=study.best_params["rf_n_estimators"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    f1_score = f1_score(y_test.to_numpy().ravel(), y_pred)
    mlflow.log_metric("test_f1", f1_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")
    mlflow.log_figure(features_correlation_plot, artifact_file="features_correlation_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="cancer_mama_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)

[I 2024-08-25 14:35:49,363] A new study created in memory with name: no-name-0d2638c1-0da7-4763-a430-2aa8913223eb
2024/08/25 14:35:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 0 at: http://localhost:5000/#/experiments/1/runs/683980ec3c334a81a32fa30947764eeb.
2024/08/25 14:35:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:52,637] Trial 0 finished with value: 0.606501376203722 and parameters: {'classifier': 'SVC_linear', 'svc_c': 6.641060475991685}. Best is trial 0 with value: 0.606501376203722.


Initial trial 0 achieved value: 0.606501376203722


2024/08/25 14:35:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 1 at: http://localhost:5000/#/experiments/1/runs/6290c3bfdbba46f498470a7ec46947fe.
2024/08/25 14:35:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:53,732] Trial 1 finished with value: 0.4883318192141721 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 4, 'rf_n_estimators': 3}. Best is trial 0 with value: 0.606501376203722.
2024/08/25 14:35:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 2 at: http://localhost:5000/#/experiments/1/runs/25f7683f2dab4561bc83073e10b8e123.
2024/08/25 14:35:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:54,500] Trial 2 finished with value: 0.42063849201718256 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 8, 'rf_n_estimators': 2}. Best is trial 0 with value: 0.

Trial 7 achieved value: 0.6101240693000993 with  0.5938% improvement


2024/08/25 14:35:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 10 at: http://localhost:5000/#/experiments/1/runs/340f7630f50a45a0a3e3ca2258f1efec.
2024/08/25 14:35:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:56,389] Trial 10 finished with value: 0.6084257042684009 and parameters: {'classifier': 'SVC_linear', 'svc_c': 1.0687932328397571}. Best is trial 7 with value: 0.6101240693000993.
2024/08/25 14:35:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 11 at: http://localhost:5000/#/experiments/1/runs/120b5ba57f374f8bb0584393d9431d77.
2024/08/25 14:35:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:56,520] Trial 11 finished with value: 0.6127735303553574 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.9961724033771858}. Best is trial 11 with value: 0.6127735303553574.
2024/

...

2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 16 at: http://localhost:5000/#/experiments/1/runs/3ef9e8146a3d4cfe86c9fb1dd6c78996.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:57,061] Trial 16 finished with value: 0.6014045467928499 and parameters: {'classifier': 'SVC_linear', 'svc_c': 0.4493364087158251}. Best is trial 13 with value: 0.6147398880732213.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 17 at: http://localhost:5000/#/experiments/1/runs/9d01cba4dbf84eefaf8089bcf5f95eb3.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:57,245] Trial 17 finished with value: 0.618487635820969 and parameters: {'classifier': 'SVC_linear', 'svc_c': 3.1015469566653753}. Best is trial 17 with value: 0.618487635820969.
2024/0

Trial 17 achieved value: 0.618487635820969 with  0.6060% improvement


2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 19 at: http://localhost:5000/#/experiments/1/runs/795e77850fda473f8aca0121e1afab1b.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:57,509] Trial 19 finished with value: 0.43496202372297443 and parameters: {'classifier': 'SVC_poly', 'svc_c': 79.01557373761062, 'svc_poly_degree': 2}. Best is trial 17 with value: 0.618487635820969.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 20 at: http://localhost:5000/#/experiments/1/runs/d05198dc5e634d8daabd6a19e15b7104.
2024/08/25 14:35:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:35:57,561] Trial 20 finished with value: 0.4856412080961417 and parameters: {'classifier': 'DecisionTreeClassifier', 'tree_max_depth': 2}. Best is trial 17 with value: 0

Trial 87 achieved value: 0.6211860485193819 with  0.4344% improvement


2024/08/25 14:36:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 90 at: http://localhost:5000/#/experiments/1/runs/9ef1a2a2fe3941e3a1388cf6c8031f17.
2024/08/25 14:36:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:36:05,883] Trial 90 finished with value: 0.5156527682843473 and parameters: {'classifier': 'SVC_poly', 'svc_c': 7.802592362595586, 'svc_poly_degree': 3}. Best is trial 87 with value: 0.6211860485193819.
2024/08/25 14:36:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 91 at: http://localhost:5000/#/experiments/1/runs/f76ebca9250d4ae6b7dd006e6e0ad0bd.
2024/08/25 14:36:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
[I 2024-08-25 14:36:06,011] Trial 91 finished with value: 0.618487635820969 and parameters: {'classifier': 'SVC_linear', 'svc_c': 3.4407419169118203}. Best is trial 87 with value: 0.6211

Una vez entrenado el modelo, en este punto se deberían realizar diferentes pruebas para validar que se encuentre en condiciones de ser puesto en producción. Una vez recibida la aprobación, se deberían ejecutar los siguientes pasos para promover el modelo.

In [23]:
client = MlflowClient()

In [24]:
# Se define el nombre del modelo y su descripción
name = "cancer_mama_model_prod"
desc = "Este clasificador realiza un diagnóstico de tumor benigno o maligno"

En caso que no sea la primera vez que se corre este notebook y ya existe un modelo en producción, se elimina el modelo anterior antes de crear el nuevo.

In [25]:
try:
    # Intenta obtener el modelo registrado por nombre
    registered_model = client.get_registered_model(name=name)
    
    # Si se encuentra, procede a eliminarlo
    client.delete_registered_model(name=name)
    print(f"El modelo '{name}' ha sido eliminado.")
    
except MlflowException as e:
    # Captura la excepción si el modelo no existe
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print(f"El modelo '{name}' no existe. No se realizó ninguna acción.")
    else:
        # Si es otra excepción, la relanza
        raise

El modelo 'rain_australia_model_prod' ha sido eliminado.


Se crea el modelo productivo en MLflow. Se le asigna el alias "champion" que indica que es el mejor hasta el momento, pero que eventualmente podría ser sustituido por otro modelo "challenger" entrenado en el dag *train_model_rain_australia* agendado en Airflow. En este modelo se almacenan como tags los metadatos del mismo así como también el resultado de la métrica *f1-score*.

In [26]:
# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2024/08/25 14:36:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rain_australia_model_prod, version 1


In [27]:
# Notificamos a la api de que existe un nuevo modelo
def notify_api_about_new_model():
    import requests

    url = "http://localhost:8800/update-model"
    response = requests.post(url)
    print(response.text)

notify_api_about_new_model()

{"status":"Sin cambios"}
